# Coastal Classificaion of sediment/non-sediment classes

In [1]:
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.4 MB/s eta 0:00:00a 0:00:01
  Using cached sankee-0.2.3-py3-none-any.whl
  Using cached eerepr-0.0.4-py3-none-any.whl (9.7 kB)
  Using cached earthengine_api-0.1.346-py3-none-any.whl
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached geocoder-1.38.1-py2.py3-none-any.whl (98 kB)
  Using cached geeadd-0.5.6-py3-none-any.whl (30 kB)
  Using cached pyperclip-1.8.2-py3-none-any.whl
  Using cached ee_extra-0.0.15-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 66.9 MB/s eta 0:00:0000:010:01m
  Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached logzero-1.7.0-py2.py3-none-any.whl (16 kB)
  Using cached plotly-5.13.1-py2.py3-none-any.whl (15.2 MB)
  Using cached ratelim-0.1.6-py2.py3-none-any.whl (4.0 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)


In [6]:
import ee
import geemap

In [21]:
js_snippet = """
//Now add the S-2 images
function maskS2clouds(image) {
  var qa = image.select('QA60');

  // Bits 10 and 11 are clouds and cirrus, respectively.
  var cloudBitMask = 1 << 10;
  var cirrusBitMask = 1 << 11;

  // Both flags should be set to zero, indicating clear conditions.
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0)
      .and(qa.bitwiseAnd(cirrusBitMask).eq(0));

  return image.updateMask(mask).divide(10000);
}

var s2Collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterDate(START_DATE, END_DATE)
                  //.filterBounds(aoi)
                  // Pre-filter to get less cloudy granules.
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))
                  //.sort('CLOUD_COVERAGE_ASSESSMENT')
                  // .sort('CLOUD_COVER')
                  .map(maskS2clouds);

                  
print(s2Collection);

//TODO: replace with a quality mosaic based on min NDWI
var s2 = s2Collection.median();

//To keep a single image, we need to do some sophisticated mosaicing
// var s2 = s2Collection.first();
print(s2);
// var s2 = s2Collection.first();

var visualization = {
  min: 0.0,
  max: 0.3,
  bands: ['B4', 'B3', 'B2'],
};


Map.addLayer(s2, visualization, 'S2-RGB');
"""

In [22]:
geemap.js_snippet_to_py(js_snippet)

In [81]:
Map = geemap.Map()

In [24]:
START_DATE = '2022-01-01'
END_DATE = '2022-12-31'

In [20]:
# Import Sentinel 1 and filter data series:
s1Collection =  (
    ee.ImageCollection('COPERNICUS/S1_GRD') \
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
      .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .filterDate(START_DATE, END_DATE)
  #.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) -> not sure if we need this \
  #.filter(ee.Filter.contains({leftField: ".geo", rightValue: aoi})) # Filter partial S1-Images of AOI
)

# print(s1Collection.limit(1)) #limit the print to the first 10 items, otherwise it will exceed the limit of 5000

#Calculate median image of time period
s1_image = ee.Image(s1Collection.median()) # convert collection to median image (i.e., median VV backscatter over time)

# Define visualization parameters in an object literal.
s1_band = 'VV'
s1_band2 = 'VH'
s1_band3 = 'angle'

vizParams = {
  'bands': [s1_band],# s1_band2, s1_band3],
  'min': -28,
  'max': 0,
}

# Display S1-Image
Map.addLayer(s1_image, vizParams, 'S1-Image')


ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.limit",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.filter",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "ImageCollection.load",
                            "arguments": {
                              "id": {
                                "constantValue": "COPERNICUS/S1_GRD"
                              }
                            }
                          }
                     

In [82]:
Map.add_basemap('SATELLITE')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [83]:
#Now add the S-2 images
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

      # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

s2Collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(START_DATE, END_DATE)
    #.filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))
    #.sort('CLOUD_COVERAGE_ASSESSMENT') \
    .map(maskS2clouds))

#TODO: replace with a quality mosaic based on min NDWI
s2 = s2Collection.median()

#To keep a single image, we need to do some sophisticated mosaicing
# s2 = s2Collection.first()
s2.getInfo()
# s2 = s2Collection.first()

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

Map.addLayer(s2, visualization, 'S2-RGB')


## Location of Interest

Let's go to Mabua Pebble beach in the Phillipines

(9.808628134071318, 125.43923549911521)

In [37]:
poi = ee.Geometry.Point([125.43923549911521,9.808628134071318])

In [84]:
Map.centerObject(poi, 15); 

# Create a composite with all bands of interest

## Grab other bands

## Modified TGSI

In [62]:

def add_mTGSI(image):
    mTGSI = image.expression('(R - B + SWIR2 - NIR) / (R + G + B + SWIR2 + NIR)', 
                             { 'R': image.select('B4'),
                              'G': image.select('B3'),
                              'B': image.select('B2'),
                              'NIR': image.select('B8'),
                              'SWIR2': image.select('B12')
                              # 'SWIR1': image.select('B11')
                            }).rename('mTGSI')
    return image.addBands(mTGSI)


def addEVI(image):
    EVI = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
    'NIR' : image.select('B8'),
    'RED' : image.select('B4'),
    'BLUE': image.select('B2')}).rename('EVI')
    
    return image.addBands(EVI)


In [68]:
viridis = ['#440154', '#3b528b', '#21918c', '#5ec962', '#fde725']
blue_yellow = ['#0000FF','#FFFF31']
blue_yellow_orange = ['#0000FF','#2c7873','#FFFF31','#f85d31']
green_blue_yellow_orange = ['#008080','#0039e6','#FFFF31','#f85d31']

vizParams_mTGSI = {'bands': ['mTGSI'], 'min': -0.45, 'max': 0.25, 'palette': green_blue_yellow_orange}

# print(Chart.image.histogram(mTGSI_s2.select('mTGSI'), aoi, 10))


In [85]:
s2 = add_mTGSI(s2)
Map.addLayer(s2.select('mTGSI'),vizParams_mTGSI,'mTGSI')

## Optional: mTGSI mask for areas

In [93]:
threshold = -0.1
sediment_bin = s2.select('mTGSI').gte(threshold).selfMask()
sedimentVisParam = {'palette': ['red']}
Map.addLayer(sediment_bin, sedimentVisParam, 'sediment')

# SNIC

In [86]:
s2

In [87]:

#Add all bands together
sentinelComp = s2.addBands(s1_image.select(s1_band))

outBands =  ["B2","B3","B4","B8","B11","B12","mTGSI","VV"]

finalComp= sentinelComp.select(outBands)
finalComp


In [69]:
js_snippet = """
var size_segmentation = 5;

// Segmentation using a SNIC approach based on the dataset previosly generated
var seeds = ee.Algorithms.Image.Segmentation.seedGrid(size_segmentation); //to get spaced grid notes at a distance specified by segmentation size parameter
var snic = ee.Algorithms.Image.Segmentation.SNIC({
  image: finalComp, //our multi-band image with selected bands same as for pixel-based
  compactness: 0,  //allow flexibility in object shape, no need to force compactness
  connectivity: 8, //use all 8 neighboring pixels in a pixel neighborhood
  neighborhoodSize: 256, 
  seeds: seeds
});
//To visualize snic result:
Map.addLayer(snic,{},'SNIC', false)


//The next step generates a list of band names from the snic image, but without "clusters" 
//since we don't need to use pixel values of their cluster IDs as a basis for class mapping:
var predictionBands=snic.bandNames().remove("clusters"); 
print(predictionBands)
 """
    
geemap.js_snippet_to_py(js_snippet)

In [90]:
snic = ee.Algorithms.Image.Segmentation.SNIC(image=s2)

In [ ]:
size_segmentation = 10

# Segmentation using a SNIC approach based on the dataset previosly generated
seeds = ee.Algorithms.Image.Segmentation.seedGrid(size_segmentation); #to get spaced grid notes at a distance specified by segmentation size parameter
snic = ee.Algorithms.Image.Segmentation.SNIC(
  image = finalComp, #our multi-band image with selected bands same as for pixel-based
  compactness = 0,  #allow flexibility in object shape, no need to force compactness
  connectivity = 8, #use all 8 neighboring pixels in a pixel neighborhood
  neighborhoodSize = 256,
  seeds = seeds
)
#To visualize snic result:
Map.addLayer(snic,{},'SNIC', False)

#The next step generates a list of band names from the snic image, but without "clusters"
#since we don't need to use pixel values of their cluster IDs as a basis for class mapping:
predictionBands=snic.bandNames().remove("clusters")
# print(predictionBands)
